In [4]:
for col in pred_df.columns:
    if col.endswith("_Prob"):
        print(f"{col}: mean={pred_df[col].mean():.3f}, min={pred_df[col].min():.2f}, max={pred_df[col].max():.2f}")


Logistic Regression_Prob: mean=0.000, min=0.00, max=0.00
SVM (RBF)_Prob: mean=0.730, min=0.73, max=0.73
XGBoost_Prob: mean=0.425, min=0.34, max=0.47
Neural Net_Prob: mean=1.000, min=1.00, max=1.00
Transformer_Prob: mean=0.578, min=0.58, max=0.58


In [ ]:
# ================================================================
# 🩺 Batch Prediction — Clean, Warning-Free & Scaled
# ================================================================
import os
import warnings
warnings.filterwarnings("ignore")

import joblib
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

# === Paths ===
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
RAW_DIR = os.path.join(BASE_DIR, "data", "raw")
PROCESSED_DIR = os.path.join(BASE_DIR, "data", "processed")
RESULTS_DIR = os.path.join(BASE_DIR, "experiments", "results")
OUTPUT_DIR = os.path.join(BASE_DIR, "data", "predictions")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# === Load combined dataset (for fitting scaler) ===
dataset_name = "combined_uci_heart"
X = joblib.load(os.path.join(PROCESSED_DIR, f"{dataset_name}_X.joblib"))
y = joblib.load(os.path.join(PROCESSED_DIR, f"{dataset_name}_y.joblib"))

# Convert sparse to dense if needed
if hasattr(X, "toarray"):
    X = X.toarray()

# Drop non-numeric columns (like 'source')
if isinstance(X, pd.DataFrame):
    non_numeric = X.select_dtypes(exclude=[np.number]).columns.tolist()
    if non_numeric:
        print(f"🧹 Dropping non-numeric columns: {non_numeric}")
        X = X.drop(columns=non_numeric)

# === Fit StandardScaler ===
scaler = StandardScaler()
scaler.fit(X)
print(f"✅ Scaler fitted on shape: {X.shape}")

# === Load new patient data ===
csv_path = os.path.join(RAW_DIR, "new_patients.csv")
if not os.path.exists(csv_path):
    raise FileNotFoundError(f"❌ Missing new patient file: {csv_path}")

new_patients = pd.read_csv(csv_path)
print(f"✅ Loaded {len(new_patients)} new patients")

# Ensure correct feature order
feature_names = [
    "age", "ca", "chol", "cp", "exang", "fbs", "oldpeak",
    "restecg", "sex", "slope", "thal", "thalach", "trestbps"
]
new_patients = new_patients[feature_names]
new_scaled = pd.DataFrame(scaler.transform(new_patients), columns=feature_names)

# === Predict with all models ===
predictions = {}

# --- Classical models ---
model_files = {
    "Logistic Regression": "logistic_regression_combined_uci_heart_model.joblib",
    "SVM (RBF)": "svm_combined_uci_heart_model.joblib",
    "XGBoost": "xgboost_combined_uci_heart_model.joblib",
}

for model_name, file in model_files.items():
    path = os.path.join(RESULTS_DIR, file)
    if os.path.exists(path):
        model = joblib.load(path)
        probs = model.predict_proba(new_scaled)[:, 1]
        predictions[model_name] = probs
    else:
        print(f"⚠️ Missing model: {model_name}")

# --- Neural Network ---
nn_path = os.path.join(RESULTS_DIR, "neural_network_combined_uci_heart_model.pt")
if os.path.exists(nn_path):
    class HeartMLP(torch.nn.Module):
        def __init__(self, input_dim):
            super().__init__()
            self.net = torch.nn.Sequential(
                torch.nn.Linear(input_dim, 64),
                torch.nn.ReLU(),
                torch.nn.Dropout(0.3),
                torch.nn.Linear(64, 32),
                torch.nn.ReLU(),
                torch.nn.Linear(32, 1),
                torch.nn.Sigmoid()
            )
        def forward(self, x):
            return self.net(x)

    nn_model = HeartMLP(len(feature_names))
    nn_model.load_state_dict(torch.load(nn_path))
    nn_model.eval()
    x_tensor = torch.tensor(new_scaled.to_numpy(), dtype=torch.float32)
    with torch.no_grad():
        nn_probs = nn_model(x_tensor).numpy().flatten()
    predictions["Neural Net"] = nn_probs
else:
    print("⚠️ Missing Neural Net model")

# --- Transformer ---
# --- Transformer (Fixed: Real Prediction Instead of Static Joblib) ---
tr_model_path = os.path.join(RESULTS_DIR, "transformer_combined_uci_heart_model.pt")

if os.path.exists(tr_model_path):
    import torch.nn as nn

    class HeartTransformer(nn.Module):
        def __init__(self, input_dim, hidden_dim=64, num_heads=2, num_layers=2, ff_dim=2048):
            super().__init__()
            self.input_proj = nn.Linear(input_dim, hidden_dim)
            encoder_layer = nn.TransformerEncoderLayer(
                d_model=hidden_dim,
                nhead=num_heads,
                dim_feedforward=ff_dim,  # 2048 matches your checkpoint
                batch_first=True
            )
            self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
            self.fc = nn.Linear(hidden_dim, 1)
            self.sigmoid = nn.Sigmoid()

        def forward(self, x):
            # Add a sequence dimension
            x = x.unsqueeze(1)
            x = self.input_proj(x)
            x = self.encoder(x)
            x = x.mean(dim=1)
            return self.sigmoid(self.fc(x))

    # Initialize and load trained weights
    transformer_model = HeartTransformer(len(feature_names))
    transformer_model.load_state_dict(torch.load(tr_model_path, map_location="cpu"))
    transformer_model.eval()

    # Prepare input tensor
    x_tensor = torch.tensor(new_scaled.to_numpy(), dtype=torch.float32)

    with torch.no_grad():
        tr_probs = transformer_model(x_tensor).numpy().flatten()

    predictions["Transformer"] = tr_probs
    print("✅ Transformer model loaded and predictions computed successfully.")
else:
    print("⚠️ Missing Transformer model weights (.pt)")


# === Combine and label ===
pred_df = new_patients.copy()
for model, probs in predictions.items():
    pred_df[f"{model}_Prob"] = probs
    pred_df[f"{model}_Pred"] = (probs >= 0.5).astype(int)
    pred_df[f"{model}_Risk"] = np.where(probs >= 0.5, "High Risk", "Low Risk")

# === Majority vote consensus ===
pred_cols = [c for c in pred_df.columns if c.endswith("_Pred")]
pred_df["Majority_Vote"] = (pred_df[pred_cols].mean(axis=1) >= 0.5).astype(int)
pred_df["Consensus_Risk"] = np.where(pred_df["Majority_Vote"] == 1, "High Risk", "Low Risk")

# === Clinician-friendly summary ===
pred_df["Summary"] = np.where(
    pred_df["Consensus_Risk"] == "High Risk",
    "Predicted HIGH risk by majority of models — further cardiac testing recommended.",
    "Predicted LOW risk by majority of models."
)

# === Save results ===
save_path = os.path.join(OUTPUT_DIR, "new_patient_predictions.csv")
pred_df.to_csv(save_path, index=False)
print(f"💾 Saved predictions to: {save_path}")

# === Visualization ===
risk_summary = {
    model: np.mean(pred_df[f"{model}_Pred"]) for model in predictions.keys()
}
risk_df = pd.DataFrame(list(risk_summary.items()), columns=["Model", "High_Risk_Rate"])

plt.figure(figsize=(7, 4))
sns.barplot(data=risk_df, x="Model", y="High_Risk_Rate", palette="coolwarm")
plt.title("Proportion of Patients Classified as High Risk per Model")
plt.ylim(0, 1)
plt.xticks(rotation=30)
plt.tight_layout()
plt.show()

display(pred_df.head(10))


🧹 Dropping non-numeric columns: ['source']
✅ Scaler fitted on shape: (916, 13)
✅ Loaded 5 new patients


RuntimeError: Error(s) in loading state_dict for HeartTransformer:
	size mismatch for encoder.layers.0.linear1.weight: copying a param with shape torch.Size([2048, 64]) from checkpoint, the shape in current model is torch.Size([128, 64]).
	size mismatch for encoder.layers.0.linear1.bias: copying a param with shape torch.Size([2048]) from checkpoint, the shape in current model is torch.Size([128]).
	size mismatch for encoder.layers.0.linear2.weight: copying a param with shape torch.Size([64, 2048]) from checkpoint, the shape in current model is torch.Size([64, 128]).
	size mismatch for encoder.layers.1.linear1.weight: copying a param with shape torch.Size([2048, 64]) from checkpoint, the shape in current model is torch.Size([128, 64]).
	size mismatch for encoder.layers.1.linear1.bias: copying a param with shape torch.Size([2048]) from checkpoint, the shape in current model is torch.Size([128]).
	size mismatch for encoder.layers.1.linear2.weight: copying a param with shape torch.Size([64